In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
#from helpers import imshow 
#from draw_matches import draw_matches

In [2]:
def siftTrack(img1,kp1,des1,img2):    
    
    #img1 = cv2.imread('demo0.png')          # queryImage
    #cam=cv2.VideoCapture(0)
    #img2 = cv2.imread('box_in_scene.jpg') # trainImage 
    #ret, QueryImgBGR=cam.read()
    
    
    MIN_MATCH_COUNT = 10
    img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    #kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2) 
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        h,w,d = img1.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
        #img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        return (1,dst)
    else:
        #print "Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT)
        matchesMask = None
        
    #################################################################################
    #draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                       #singlePointColor = None,
                       #matchesMask = matchesMask, # draw only inliers
                       #flags = 2)
    #img4 = QueryImgBGR.copy()
    #img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,img4,**draw_params)
    #plt.imshow(img3, 'gray'),plt.show()
    #################################################################################
    return (-1,None)

In [ ]:
cam=cv2.VideoCapture(0)
img1 = cv2.imread('demo0.png')
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1,None)
while True:
    ret, img2=cam.read()
    #checker,dst = siftTrack(img1,kp1,des1,img2)
    #if checker != -1:
    #    img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    #    print(checker)
    cv2.imshow('mask',img2)
cam.release()
cv2.destroyAllWindows()

In [10]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
def siftTrack2(trainImg,trainKP,trainDesc,QueryImgBGR):
    MIN_MATCH_COUNT=10

    detector=cv2.xfeatures2d.SIFT_create()

    FLANN_INDEX_KDITREE=0
    flannParam=dict(algorithm=FLANN_INDEX_KDITREE,tree=5)
    searchParam = dict(checks=50)
    flann=cv2.FlannBasedMatcher(flannParam,searchParam)

    #trainImg=cv2.imread("demo0.png",0)
    #trainKP,trainDesc=detector.detectAndCompute(trainImg,None)

    #cam=cv2.VideoCapture(0)
    #while True:
    #ret, QueryImgBGR=cam.read()
    
    QueryImg=cv2.cvtColor(QueryImgBGR,cv2.COLOR_BGR2GRAY)
    queryKP,queryDesc=detector.detectAndCompute(QueryImg,None)
    matches=flann.knnMatch(queryDesc,trainDesc,k=2)

    goodMatch=[]
    for m,n in matches:
        if(m.distance < 0.7*n.distance):
            goodMatch.append(m)
    if(len(goodMatch) > MIN_MATCH_COUNT):
        tp=[]
        qp=[]
        for m in goodMatch:
            tp.append(trainKP[m.trainIdx].pt)
            qp.append(queryKP[m.queryIdx].pt)
        tp,qp=np.float32((tp,qp))
        H,status=cv2.findHomography(tp,qp,cv2.RANSAC,3.0)
        h,w=trainImg.shape
        trainBorder=np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]])
        queryBorder=cv2.perspectiveTransform(trainBorder,H)
        cv2.polylines(QueryImgBGR,[np.int32(queryBorder)],True,(0,255,0),5)
        return (1,queryBorder)
        #print(queryBorder)
    else:
        print("Not Enough match found")
        return (-1,None)



In [11]:
cam = cv2.VideoCapture(0)
img1 = cv2.imread("demo0.png")
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1,None)
while True:
    ret, img2=cam.read()
    checker,dst = siftTrack2(img1,kp1,des1,img2)
    if checker != -1:
        img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        print(checker)
    cv2.imshow('mask',img2)
cam.release()
cv2.destroyAllWindows()

ValueError: too many values to unpack (expected 2)